<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Sizing of a multi-rotor drone

*Written by Marc Budinger (INSA Toulouse), Scott Delbecq (ISAE-SUPAERO) and Félix Pollet (ISAE-SUPAERO), Toulouse, France.*

The objective of this notebook is to select the best compromise of components (propeller, motor, ESC, battery) of a multi-rotor drone for given specifiations.

In [1]:
# Import libraries
import math
import numpy as np
from utils.openmdao_generator import GUI_notebook

## Sizing code

The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable.   
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue   
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

Concerning overconstraints components, we have here:
- Brushless motors with multiple torque and voltage constraints (hover and transient vertical displacement) 

Multiple algebraic loops appears in the sizing problem:
- The thrust depends of the total mass which depend of components required for generating this thrust

The final optimization problem depends thus of these parameters:
- $\beta_{pro}=pitch/diameter$ ratio to define the propeller
- $k_{os}$ over sizing coefficient on the load mass to estimate the final total mass
- $k_{mot}$ over sizing coeffcient on the motor torque to estimate the max torque with the hover flight conditions
- $k_{speed,mot}$ over sizing coeffcient on the motor speed to take into account voltage limits during hover or take-off flight
- $k_{ND}$ slow down propeller coef : ND = kNDmax / k_ND
- $k_{D}$ aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame
- $k_{mb}$ ratio battery mass / payload mass
- $k_{vb}$ over sizing coefficient for the battery voltage

More details in the setting up of sizing code can be found in the  [following paper](https://www.researchgate.net/profile/Marc_Budinger/publication/277933677_Computer-aided_definition_of_sizing_procedures_and_optimization_problems_of_mechatronic_systems/links/55969de508ae793d137c7ea5/Computer-aided-definition-of-sizing-procedures-and-optimization-problems-of-mechatronic-systems.pdf):  

> Reysset, A., Budinger, M., & Maré, J. C. (2015). Computer-aided definition of sizing procedures and optimization problems of mechatronic systems. Concurrent Engineering, 23(4), 320-332.

The sizing code is defined here in a function which can give:
- an evaluation of the objective: here the total mass
- an evaluation of the constraints: 

Here is an non-exhaustive XDSM diagram of the multirotor sizing code:

![XDSM](pictures/xdsm_multirotor_mdo.png)

We will now build the sizing code for the system optimization. 

### 1. Specifications
The first step is to provide the specifications (top-level requirements) for the drone.

Main specifications :
- a load (video, control card) of mass $M_{load}$.  
- an autonomy $t_{hf}$ for the hover flight.
- an acceleration to take off $a_{to}$.

In [2]:
# SPECIFICATIONS

# Load
M_pay = 50. # [kg] load mass

# Acceleration during take off
a_to = 0.25 * 9.81 # [m/s**2] acceleration

# Autonomy
t_hov_spec = 25. # [min] time of hover flight

# MTOW
MTOW = 360. # [kg] maximal mass allowed

### 2. Architecture definition and design assumptions

Then, we must provide the architecture definition and design assumptions for the models.

In [3]:
# ARCHITECTURE of the multi-rotor drone (4,6, 8 arms, ...)
N_arm = 4 # [-] number of arms
N_pro_arm = 1 # [-] number of propeller per arm (1 or 2)

# BATTERY AND ESC : reference parameters for scaling laws
# Ref : MK-quadro
M_bat_ref = .329 # [kg] mass
E_bat_ref = 220.*3600.*.329 # [J]
C_bat_ref= 5 # [Ah] Capacity
I_bat_max_ref = 50*C_bat_ref # [A] max discharge current

# Ref : Turnigy K_Force 70HV 
P_esc_ref = 3108. # [W] Power
M_esc_ref = .115 # [kg] Mass

# MOTOR : reference parameters for scaling laws
# Ref : AXI 5325/16 GOLD LINE
T_nom_mot_ref = 2.32  # [N*m] rated torque
T_max_mot_ref = 85./70.*T_nom_mot_ref # [N*m] max torque
R_mot_ref = 0.03  # [ohm] resistance
M_mot_ref = 0.575 # [kg] mass
K_T_ref = 0.03 # [N*m/A] torque coefficient
T_mot_fr_ref = 0.03 # [N*m] friction torque (zero load, nominal speed)

# FRAME
sigma_max = 280e6/4. # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
rho_s = 1700. # [kg/m**3] Volumic mass of aluminum

# PROPELLER
# Specifications
rho_air=1.18# [kg/m**3] Air density
ND_max=105000./60.*.0254 #[Hz.m] Max speed limit (N.D max) for APC MR propellers

# Reference parameters for scaling laws
D_pro_ref=11.*.0254# [m] Reference propeller diameter
M_pro_ref=0.53*0.0283# [kg] Reference propeller mass

### 3. Optimization variables

The next step is to give an initial value for the optimisation variables:

In [4]:
# Optimisation variables : initial values
beta_pro = .33 # pitch/diameter ratio of the propeller
k_os = 3.2 # over sizing coefficient on the load mass 
k_ND = 1.2 # slow down propeller coef : ND = NDmax / k_ND
k_mot = 1. # over sizing coefficient on the motor torque
k_speed_mot = 1.2 # adaption winding coef on the motor speed
k_mb = 1. # ratio battery/load mass
k_vb = 1. # oversizing coefficient for voltage evaluation
k_D = .99 # aspect ratio D_in/D_out for the beam of the frame

### 4. Sizing code

Now, complete the following sizing code with the missing equations:

In [8]:
#%% DRONE

#% SCENARIOS
# ---
M_total = k_os * M_pay  # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)
N_pro = N_pro_arm * N_arm  # Propellers number
F_pro_hov = M_total * (9.81) / N_pro  # [N] Thrust per propeller for hover
F_pro_to = M_total * (9.81 + a_to) / N_pro  # [N] Thrust per propeller for take-off


#% PROPELLER
# --- 
# Estimation models for propeller aerodynamics
C_t = 4.27e-02 + 1.44e-01 * beta_pro  # Thrust coef with T=C_T.rho.n^2.D^4
C_p = -1.48e-03 + 9.72e-02 * beta_pro  # Power coef with P=C_p.rho.n^3.D^5

# Propeller selection with take-off scenario

nD = ND_max/k_ND
D_pro = math.sqrt(F_pro_to/(C_t * rho_air * (ND_max/k_ND**2)))  # [m] Propeller diameter
n_pro_to = nD/D_pro # [Hz] Propeller speed 
Omega_pro_to = (2*math.pi/60)*n_pro_to  # [rad/s] Propeller speed

# Estimation model for mass
M_pro = M_pro_ref * (D_pro/D_pro_ref) # [kg] Propeller mass

# Performance in various operating conditions
# Take-off
P_pro_to = C_p*rho_air*(nD)**3*D_pro**2 # [W] Power per propeller
T_pro_to = P_pro_to/Omega_pro_to # [N*m] Propeller torque
# Hover
n_pro_hov = math.sqrt(F_pro_hov/C_t * rho_air * D_pro**4) # [Hz] hover speed
Omega_pro_hov = (2*math.pi/60)*n_pro_hov # [rad/s] Propeller speed
P_pro_hov = C_p*rho_air*(n_pro_hov)**3*D_pro**5 # [W] Power per propeller
T_pro_hov = P_pro_hov/Omega_pro_hov # [N*m] Propeller torque   


#% MOTOR
# --- 
# Nominal torque selection with hover scenario
T_nom_mot = k_mot * T_pro_hov # [N*m] Motor nominal torque per propeller

# Torque constant selection with take-off scenario
U_bat = k_vb*1.84*P_pro_to**(0.36)  # [V] battery voltage estimation
K_T = U_bat/(k_speed_mot * Omega_pro_to) # [N*m/A] or [V/(rad/s)] Kt motor

# Estimation models
M_mot = M_mot_ref * (T_nom_mot/T_nom_mot_ref)**(3/3.5) # [kg] Motor mass
R_mot = R_mot_ref * (K_T/K_T_ref)**2 * (T_nom_mot/T_nom_mot_ref)**(-5/3.5)  # [ohm] motor resistance
T_mot_fr = T_mot_fr_ref * (T_nom_mot/T_nom_mot_ref)**(3/3.5)  # [N*m] Friction torque
T_max_mot = T_max_mot_ref * (T_nom_mot/T_nom_mot_ref)  # [N*m] Max. torque

# Performance in various operating conditions
# Hover current and voltage
I_mot_hov = T_pro_hov/K_T  # [A] Current of the motor per propeller
U_mot_hov = K_T * Omega_pro_hov + R_mot * I_mot_hov  # [V] Voltage of the motor per propeller
P_el_mot_hov = U_mot_hov * I_mot_hov  # [W] Hover : electrical power
# Takeoff current and voltage
I_mot_to = T_pro_to/K_T  # [A] Current of the motor per propeller
U_mot_to = K_T * Omega_pro_to + R_mot * I_mot_to # [V] Voltage of the motor per propeller
P_el_mot_to = U_mot_to * I_mot_to # [W] Takeoff : electrical power


#% BATTERY
# ---     
# Voltage selection with takeoff scenario
# U_bat = k_vb*1.84*P_pro_to**(0.36)  # [V] battery voltage estimation

# Energy selection with payload mass
M_bat = k_mb * M_pay # [kg] Battery mass
E_bat = E_bat_ref * (M_bat/M_bat_ref) # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)

# Estimation models
C_bat = E_bat/U_bat # [A*s] Capacity  of the battery 
I_bat_max = I_bat_max_ref * (C_bat/C_bat_ref) # [A] Max discharge current
P_bat_max = U_bat * I_bat_max  # [W] Max power

# Performance in hover
I_bat_hov = P_el_mot_hov/U_bat  # [A] Current of the battery


#% ESC
# ---
# Power selection with takeoff scenario
P_esc = I_mot_to * U_bat  # [W] power electronic power (corner power or apparent power)

# Estimation models
U_esc = 1.84*P_esc**(0.36)  # [V] ESC voltage     
M_esc = M_esc_ref * (P_esc/P_esc_ref)  # [kg] Mass ESC
   

#% FRAME
# ---
# Arms selection with propellers size
alpha_sep = math.pi/N_arm # [rad] interior angle separation between propellers
L_arm = D_pro/(2*math.sin(alpha_sep))  # [m] length of the arm

# Tube diameter & thickness selection with take-off scenario
D_in_arm = ((F_pro_to * L_arm * 32 * k_D)/(math.pi * sigma_max * (k_D**4 - 1)))**(1/3) # [m] inner diameter of the arm (hollow cylinder)
D_out_arm = D_in_arm/k_D  # [m] outer diameter of the arm (hollow cylinder)

# Estimation models
M_arms = math.pi/4 * (D_out_arm**2 - D_in_arm**2) * rho_s * L_arm * N_arm  # [kg] mass of the arms
M_body = (1/0.4) * M_arms # [kg] mass of the body (40% of total mass is the arms)
M_frame = M_body + M_arms  # [kg] total mass of the frame


#% OBJECTIVES
# ---
t_hov = C_bat/I_mot_hov # [min] Hover time 
M_total_real = (M_pro * N_pro) + (M_mot * N_arm) + (M_bat) + (M_esc * N_arm) + (M_frame) # [kg] Total mass


#% CONSTRAINTS
cons_1 = M_total-M_total_real
cons_2 = T_max_mot-T_pro_to
cons_3 = U_bat-U_mot_to
cons_4 = P_bat_max-(P_el_mot_to*N_pro)/.95
cons_5 = U_esc-U_bat
cons_6 = t_hov-t_hov_spec
cons_7 = MTOW-M_total_real

IndentationError: unexpected indent (3102614755.py, line 33)

If the code runs correctly (no error when executing the cell), you can now generate an OpenMDAO model from your equations. OpenMDAO is an open-source optimization framework that will allow you to evaluate and optimize the multi-rotor drone (see next notebook).

To generate your OpenMDAO model, copy and paste your equations in the following widget. Then click on "Analyse" and wait for the analysis to be completed. The column "variable name" proposes a new notation for the variables, which will be useful later. You may also complete the "Units" column, though this task is not mandatory.

If your fine with the analysis, click on the "Generate File" button. This will create a "DRONE.py" file containing your sizing code in the OpenMDAO format. You can now move on to the next (and last!) notebook.

In [6]:
vb = GUI_notebook.init(In)
vb

Box(children=(Box(children=(Btn(children=['Copy cells'], color='blue lighten-1', height='35px', width='250px')…